In [20]:
import numpy as np
from scipy.io import wavfile
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# If you previously hit nbformat issues, ensure nbformat is installed in the same env as Jupyter:
#   pip install -U nbformat

import plotly.io as pio
# Best default for JupyterLab/Notebook; if it still doesn't render, try: "browser"
pio.renderers.default = "notebook_connected"


In [21]:
def load_wav_mono(path):
    fs, x = wavfile.read(path)

    # Convert to float32 in [-1, 1]
    if x.dtype == np.int16:
        x = x.astype(np.float32) / 32768.0
    elif x.dtype == np.int32:
        x = x.astype(np.float32) / 2147483648.0
    else:
        x = x.astype(np.float32)

    # Mono
    if x.ndim > 1:
        x = x[:, 0]

    # Remove DC offset (helpful)
    x = x - np.mean(x)

    return fs, x


In [22]:
def stft_mag(x, fs, win=2048, hop=512):
    """Return (freqs, times, mag) where mag shape is [frames, bins]."""
    w = np.hanning(win).astype(np.float32)
    n_frames = 1 + (len(x) - win) // hop
    if n_frames <= 0:
        raise ValueError("Signal too short for given win/hop.")

    frames = np.empty((n_frames, win), dtype=np.float32)
    for i in range(n_frames):
        start = i * hop
        frames[i] = x[start:start+win] * w

    X = np.fft.rfft(frames, axis=1)
    mag = np.abs(X).astype(np.float32)  # linear magnitude (not dB)
    freqs = np.fft.rfftfreq(win, d=1/fs)
    times = (np.arange(n_frames) * hop) / fs
    return freqs, times, mag


def band_energy(mag, freqs, fmin, fmax, power=True):
    """
    mag: [frames, bins], freqs: [bins]
    returns per-frame band energy
    """
    idx = (freqs >= fmin) & (freqs <= fmax)
    if not np.any(idx):
        raise ValueError(
            f"No FFT bins in band {fmin}-{fmax} Hz. Increase win or adjust band.")
    if power:
        return np.mean((mag[:, idx] ** 2), axis=1)  # mean power
    return np.mean(mag[:, idx], axis=1)            # mean magnitude


def transient_rms_blocks(x, fs, block=1024):
    """
    Compute RMS of first difference per block (matches your C++ starter approach).
    Returns (times, rms)
    """
    y = np.diff(x, prepend=x[0])
    n_blocks = len(y) // block
    y = y[:n_blocks * block].reshape(n_blocks, block)
    rms = np.sqrt(np.mean(y * y, axis=1) + 1e-12)
    peak = np.max(np.abs(y), axis=1)
    t = (np.arange(n_blocks) * block) / fs
    return t, rms, peak

In [23]:
def compute_features(x, fs,
                     stft_win=2048, stft_hop=512,
                     low_band=(200, 2000),
                     high_band=(4000, 12000),
                     transient_block=1024):
    freqs, t_stft, mag = stft_mag(x, fs, win=stft_win, hop=stft_hop)

    low = band_energy(mag, freqs, low_band[0], low_band[1], power=True)
    high = band_energy(mag, freqs, high_band[0], high_band[1], power=True)
    ratio = high / (low + 1e-12)

    t_tr, tr_rms, peakiness = transient_rms_blocks(x, fs, block=transient_block)

    return {
        "freqs": freqs,
        "t_stft": t_stft,
        "mag": mag,
        "low": low,
        "high": high,
        "ratio": ratio,
        "t_tr": t_tr,
        "tr_rms": tr_rms,
        "peakiness": peakiness,
        "params": {
            "stft_win": stft_win,
            "stft_hop": stft_hop,
            "low_band": low_band,
            "high_band": high_band,
            "transient_block": transient_block,
        }
    }


In [24]:
def plot_waveform_and_spectrogram(x, fs, feats, title=""):
    t = np.arange(len(x)) / fs

    # Spectrogram in dB
    S_db = 20*np.log10(feats["mag"].T + 1e-12)  # [bins, frames]

    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=False,
        row_heights=[0.35, 0.65],
        vertical_spacing=0.12,
        subplot_titles=("Waveform", "Spectrogram (dB)")
    )

    fig.add_trace(go.Scatter(x=t, y=x, mode="lines", name="x", line=dict(width=1)), row=1, col=1)

    fig.add_trace(go.Heatmap(
        x=feats["t_stft"],
        y=feats["freqs"],
        z=S_db,
        colorscale="Turbo",
        colorbar=dict(title="dB")
    ), row=2, col=1)

    fig.update_yaxes(title_text="Amplitude", row=1, col=1)
    fig.update_yaxes(title_text="Hz", range=[0, 12000], row=2, col=1)
    fig.update_xaxes(title_text="Time (s)", row=2, col=1)

    fig.update_layout(
        title=title,
        template="plotly_white",
        height=700
    )
    fig.show()


In [25]:
def detect_hits_from_features(feats,
                              high_thresh,
                              tr_thresh,
                              ratio_thresh,
                              refractory_s=0.07):
    """
    Returns list of hit times (seconds) and a dict of per-frame boolean masks.
    Uses STFT timebase for high/low/ratio; uses transient RMS timebase for tr_rms.
    We align by time (nearest neighbor).
    """
    t_stft = feats["t_stft"]
    high = feats["high"]
    ratio = feats["ratio"]

    t_tr = feats["t_tr"]
    tr = feats["tr_rms"]
    p = feats["peakiness"]

    # Map each STFT frame to nearest transient block index
    tr_idx = np.searchsorted(t_tr, t_stft, side="right") - 1
    tr_idx = np.clip(tr_idx, 0, len(tr) - 1)
    tr_aligned = tr[tr_idx]
    p_aligned = p[tr_idx]

    gate_high = high > high_thresh
    gate_tr = tr_aligned > tr_thresh
    gate_ratio = ratio > ratio_thresh

    gate_all = gate_high & gate_tr & gate_ratio

    hits = []
    last_t = -1e9
    for i, ok in enumerate(gate_all):
        if not ok:
            continue
        ti = float(t_stft[i])
        if (ti - last_t) >= refractory_s:
            hits.append(ti)
            last_t = ti

    masks = {
        "gate_high": gate_high,
        "gate_tr": gate_tr,
        "gate_ratio": gate_ratio,
        "gate_all": gate_all,
        "tr_aligned": tr_aligned,
        "p_aligned": p_aligned,
    }
    return hits, masks


In [26]:
def plot_features_with_hits(feats, masks, hits, title="Features"):
    t = feats["t_stft"]

    fig = make_subplots(
        rows=5, cols=1, shared_xaxes=True,
        vertical_spacing=0.06,
        subplot_titles=(
            "High-band energy (power)",
            "Low-band energy (power)",
            "High/Low ratio",
            "Transient RMS (aligned to STFT frames)",
            "Peakiness",
        )
    )

    fig.add_trace(go.Scatter(
        x=t, y=feats["high"], mode="lines", name="high", line=dict(width=1)), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=t, y=feats["low"],  mode="lines", name="low",  line=dict(width=1)), row=2, col=1)
    fig.add_trace(go.Scatter(
        x=t, y=feats["ratio"], mode="lines", name="ratio", line=dict(width=1)), row=3, col=1)
    fig.add_trace(go.Scatter(
        x=t, y=masks["tr_aligned"], mode="lines", name="tr_rms", line=dict(width=1)), row=4, col=1)
    fig.add_trace(go.Scatter(
        x=t, y=masks["p_aligned"], mode="lines", name="peakiness", line=dict(width=1)), row=5, col=1)

    # Add hit markers
    for th in hits:
        for r in range(1, 6):
            fig.add_vline(x=th, line_dash="dash", line_width=1, row=r, col=1)

    fig.update_layout(
        title=title,
        template="plotly_white",
        height=900
    )
    fig.update_xaxes(title_text="Time (s)", row=4, col=1)
    fig.show()

In [27]:
fs, x = load_wav_mono("test.wav")

feats = compute_features(
    x, fs,
    stft_win=2048, stft_hop=512,
    low_band=(200, 2000),
    high_band=(4000, 12000),
    transient_block=1024
)

# plot_waveform_and_spectrogram(x, fs, feats, title="strike_01.wav")

# Initial thresholds (these are starting points; you'll tune them)
high_thresh = np.median(feats["high"]) * 10.0
tr_thresh   = np.median(feats["tr_rms"]) * 10.0
ratio_thresh = 0.55

hits, masks = detect_hits_from_features(
    feats,
    high_thresh=0.5,
    tr_thresh=0.02,
    ratio_thresh=0.4,
    refractory_s=0.13
)

print("Hits:", hits)
plot_features_with_hits(feats, masks, hits, title="Features + detected hits (strike_01)")


Hits: [23.445333333333334, 39.70133333333333, 44.629333333333335]


In [40]:
# One-cell Jupyter implementation: High-pass filter → block RMS/peakiness → adaptive threshold → hits → Plotly chart
import numpy as np
from scipy.io import wavfile
from scipy.signal import butter, sosfilt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# ---------- Config ----------
WAV_PATH = "test.wav"   # change if needed

FS_TARGET = 48000                # informational (we use the file's fs)
HP_CUTOFF_HZ = 8000              # high-pass cutoff
HP_ORDER = 4                     # 4th order (sos, stable)

BLOCK = 2048
REFRACTORY_S = 0.12              # prevents double-fires
NOISE_ALPHA = 0.01               # noise floor EMA smoothing
MULT = 8.0                       # threshold multiplier over noise floor
MIN_THRESH = 0.03              # absolute floor on RMS (tune)
PEAKINESS_MIN = 8.0              # optional; raise to reduce false positives
USE_PEAKINESS_GATE = True        # set False to use RMS-only gate

# ---------- Load WAV (mono float) ----------
fs, x = wavfile.read(WAV_PATH)

if x.dtype == np.int16:
    x = x.astype(np.float32) / 32768.0
elif x.dtype == np.int32:
    x = x.astype(np.float32) / 2147483648.0
else:
    x = x.astype(np.float32)

if x.ndim > 1:
    x = x[:, 0]

x = x - np.mean(x)  # remove DC
duration_s = len(x) / fs

# ---------- High-pass filter ----------
sos = butter(HP_ORDER, HP_CUTOFF_HZ, btype="highpass", fs=fs, output="sos")
x_hp = sosfilt(sos, x)

# ---------- Block features: RMS + Peakiness ----------
n_blocks = len(x_hp) // BLOCK
x_hp_b = x_hp[:n_blocks * BLOCK].reshape(n_blocks, BLOCK)

rms_hp = np.sqrt(np.mean(x_hp_b * x_hp_b, axis=1) + 1e-12)
peak_hp = np.max(np.abs(x_hp_b), axis=1) + 1e-12
peakiness = peak_hp / (rms_hp + 1e-12)

t = (np.arange(n_blocks) * BLOCK) / fs  # block timebase

# ---------- Adaptive threshold + hit detection ----------
noise = 0.0
thr = np.zeros_like(rms_hp)
gate_rms = np.zeros_like(rms_hp, dtype=bool)
gate_pk = np.zeros_like(rms_hp, dtype=bool)
gate_all = np.zeros_like(rms_hp, dtype=bool)

hits = []
last_hit_t = -1e9

for i in range(n_blocks):
    ti = float(t[i])

    # update noise floor only when "far" from a hit (prevents learning the hit)
    if (ti - last_hit_t) > 0.25:
        noise = rms_hp[i] if noise == 0.0 else (
            1.0 - NOISE_ALPHA) * noise + NOISE_ALPHA * rms_hp[i]

    thr_i = max(MIN_THRESH, MULT * noise)
    thr[i] = thr_i

    gate_r = rms_hp[i] > thr_i
    gate_p = peakiness[i] > PEAKINESS_MIN
    gate_rms[i] = gate_r
    gate_pk[i] = gate_p

    ok = gate_r and ((not USE_PEAKINESS_GATE) or gate_p)

    if ok and (ti - last_hit_t) >= REFRACTORY_S:
        hits.append(ti)
        last_hit_t = ti
        gate_all[i] = True

# ---------- Plotly chart (same spirit as plot_features_with_hits) ----------
pio.renderers.default = "notebook_connected"  # if issues, try: "browser"

fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.06,
    subplot_titles=(
        f"High-pass RMS per block (HP>{HP_CUTOFF_HZ} Hz) with Adaptive Threshold (mult={MULT}, min={MIN_THRESH})",
        f"Peakiness per block (peak/RMS) (min={PEAKINESS_MIN}, enabled={USE_PEAKINESS_GATE})",
        "Gate / Hits"
    )
)

# Row 1: RMS + threshold
fig.add_trace(go.Scatter(x=t, y=rms_hp, mode="lines",
              name="rms_hp", line=dict(width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=thr, mode="lines", name="threshold",
              line=dict(width=1, dash="dash")), row=1, col=1)

# Row 2: Peakiness
fig.add_trace(go.Scatter(x=t, y=peakiness, mode="lines",
              name="peakiness", line=dict(width=1)), row=2, col=1)
fig.add_hline(y=PEAKINESS_MIN, line_dash="dash", row=2, col=1)

# Row 3: show gates as 0/1 signals (helpful visually)
fig.add_trace(go.Scatter(x=t, y=gate_rms.astype(int), mode="lines",
              name="gate_rms", line=dict(width=1)), row=3, col=1)
if USE_PEAKINESS_GATE:
    fig.add_trace(go.Scatter(x=t, y=gate_pk.astype(int), mode="lines",
                  name="gate_peakiness", line=dict(width=1)), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=gate_all.astype(int),
              mode="markers", name="HIT", marker=dict(size=6)), row=3, col=1)

# Add vertical lines for hits across all rows
for th in hits:
    for r in (1, 2, 3):
        fig.add_vline(x=th, line_dash="dash", line_width=1, row=r, col=1)

fig.update_yaxes(title_text="RMS", row=1, col=1)
fig.update_yaxes(title_text="Peak/RMS", row=2, col=1)
fig.update_yaxes(title_text="Gate", row=3, col=1, range=[-0.1, 1.1])
fig.update_xaxes(title_text="Time (s)", row=3, col=1)

fig.update_layout(
    title=f"StrikePoint: HP-filtered RMS detector on {WAV_PATH.split('/')[-1]} (fs={fs} Hz, duration={duration_s:.1f}s)",
    template="plotly_white",
    height=900,
    legend=dict(orientation="h", yanchor="bottom",
                y=1.02, xanchor="left", x=0.0)
)

print(f"Detected hits ({len(hits)}): {hits}")
fig.show()

Detected hits (5): [14.421333333333333, 23.424, 33.664, 44.629333333333335, 56.40533333333333]
